In [30]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [31]:
import ee
import pandas as pd
import gee_utils

import math

In [44]:
# Trigger the authentication flow.
ee.Authenticate()


Successfully saved authorization token.


In [32]:
# Initialize the library.
ee.Initialize()

In [37]:
EXPORT = 'drive'
BUCKET = None

LSMS_EXPORT_FOLDER = 'lsms_tfrecords_raw'

In [58]:
LSMS_CSV_PATH = '../data/lsms_clusters.csv'

MS_BANDS = ['BLUE', 'GREEN', 'RED', 'NIR', 'SW_IR1', 'SW_IR2', 'TEMP']

PROJECTION = 'EPSG:3857'
SCALE = 30              
EXPORT_TILE_RADIUS = 127  # image dimension = (2*EXPORT_TILE_RADIUS) + 1 = 255px
CHUNK_SIZE = 25         # set to a small number (<= 50) if Google Earth Engine reports memory errors

In [59]:
def export_images(df, country, export_folder, chunk_size):
    num_chunks = int(math.ceil(len(df) / chunk_size))
    tasks = {}

    for i in range(num_chunks):
        chunk_slice = slice(i * chunk_size, (i+1) * chunk_size - 1)
        fc = gee_utils.create_fc(df.loc[chunk_slice, :])

        roi = fc.geometry()
        composite_img = gee_utils.create_ms_img(roi).select(MS_BANDS)

        composite_img = gee_utils.add_latlon_band(composite_img)
        composite_img = gee_utils.add_nl_band(composite_img)
                
        fname = f'{country}_{i:02d}'
        tasks[(export_folder, country, i)] = gee_utils.make_tiles(
            img=composite_img, scale=SCALE, ksize=EXPORT_TILE_RADIUS,
            points=fc, export='drive',
            prefix=export_folder, fname=fname,
            bucket=None)
    return tasks

In [60]:
df_lsms_eth = pd.read_csv(LSMS_CSV_PATH)
new_task = export_images(df_lsms_eth, country="ethiopia", export_folder=LSMS_EXPORT_FOLDER, chunk_size=CHUNK_SIZE)